# Appendix A.2: Computational Cost

This notebook reproduces the analyses in the Appendix, Section A.2 of the manuscript, including Tables 1-3.

> **Note:** By default, the computational cost experiments are run in parallel on 6 cores. If you have less cores available on your system, you should change that number according to your resources: `addprocs(5)` means that 5 processes on separates cores are added to the one being used already, so if you want to use, e.g., 4 cores, change that to `addprocs(3)`. If you do not want to run the code in parallel or have no sufficient resources to do to, you can comment out the line altogether and the code will run without further changes on a single core. 

## Setup 

First, we set up the parallel computing: 

In [14]:
using Distributed
addprocs(5)

5-element Vector{Int64}:
  7
  8
  9
 10
 11

Next, we load the Julia environment specified in the `Project.toml` and `Manifest.toml` files: First, we activate this environment, then install all dependencies (if some packages are not yet installed), and print out which packages and versions are currently in our environment. To make it available on all used processes, we use the `@everywhere` macro.

In [15]:
@everywhere using Pkg; 

# all paths are relative to the `notebook` subfolder main folder, i.e., assuming `pwd()` outputs
# ".../DeepDynamicodelingWithJust2TimePoints/notebooks"

@everywhere Pkg.activate("../.")
Pkg.instantiate()

      From worker 10:	  Activating environment at `~/Desktop/DeepDynamicModelingWithJust2TimePoints/Project.toml`
      From worker 8:	  Activating environment at `~/Desktop/DeepDynamicModelingWithJust2TimePoints/Project.toml`
      From worker 11:	  Activating environment at `~/Desktop/DeepDynamicModelingWithJust2TimePoints/Project.toml`
      From worker 9:	  Activating environment at `~/Desktop/DeepDynamicModelingWithJust2TimePoints/Project.toml`
      From worker 7:	  Activating environment at `~/Desktop/DeepDynamicModelingWithJust2TimePoints/Project.toml`
      From worker 5:	  Activating environment at `~/Desktop/DeepDynamicModelingWithJust2TimePoints/Project.toml`
      From worker 2:	  Activating environment at `~/Desktop/DeepDynamicModelingWithJust2TimePoints/Project.toml`
      From worker 6:	  Activating environment at `~/Desktop/DeepDynamicModelingWithJust2TimePoints/Project.toml`
      From worker 3:	  Activating environment at `~/Desktop/DeepDynamicModelingWithJust2TimePo

  Activating environment at `~/Desktop/DeepDynamicModelingWithJust2TimePoints/Project.toml`


Precompiling 

project...


  ✓ BenchmarkTools


  ✓ Plots


  2 dependencies successfully precompiled in 33 seconds (278 already precompiled)


Next, we load and precompile the necessary packages (in the versions specified by the `*.toml` files). 

In [16]:
@everywhere using BenchmarkTools
@everywhere using DataFrames
@everywhere using Distributed
@everywhere using Distributions
@everywhere using Random
@everywhere using Flux
@everywhere using DiffEqFlux
@everywhere using OrdinaryDiffEq
@everywhere using SharedArrays

Additionally, we import some user-defined functions, with different files for separate functionality, also using `@everywhere` to define them on all procs. 

In [17]:
@everywhere include("../src/simulation.jl") # for simulating data
@everywhere include("../src/model.jl") # for initializing and training the model
@everywhere include("../src/benchmarking.jl") # for plotting data and learned trajectories

## Define ground truth developments

First, we define the ground-truth developments as solutions of the underlying two-dimensional linear ODE system with two distinct sets of parameters, corresponding to two groups of individuals with two distinct underlying development patterns. 

In [18]:
# define initial condition
@everywhere true_u0 = Float32[2, 1]
# define time span on which to solve the ODE
@everywhere tspan = (0.0f0, 10.0f0)
# define parameters for the two distinct groups
@everywhere true_odeparams_group1 = Float32[-0.2, 0.00, 0.00, -0.2]
@everywhere true_odeparams_group2 = Float32[-0.2, 0.00, 0.00, 0.2]
  
# define corresponding ODE problems for the two groups
@everywhere prob1 = ODEProblem(linear_2d_system,true_u0,tspan,true_odeparams_group1)
@everywhere prob2 = ODEProblem(linear_2d_system,true_u0,tspan,true_odeparams_group2)
  
# solve ODE systems to obtain "true" underlying trajectory in each group
@everywhere dt=0.1
@everywhere sol_group1 = solve(prob1, Tsit5(), saveat = dt);
@everywhere sol_group2 = solve(prob2, Tsit5(), saveat = dt);

## Train model using benchmarking

Now, we train the model on varying numbers of individuals, time-dependent variables and baseline variables. We save all benchmark results, i.e., runtime, memory, and allocations, in a specific `SharedArray`, an array that allows for being used simultaneous by different processes while preventing them from getting in the way of each other. 

### Setup

In [19]:
# define number of observations, variables and baseline variables to try 
@everywhere n_obs = [50, 100, 250, 500, 1000, 2000, 5000]
@everywhere n_vars = [10, 20, 50, 100, 200]
@everywhere n_baselinevars = [10, 20, 50, 100, 200]
@everywhere lenobs, lenvars, lenbvars = length(n_obs), length(n_vars), length(n_baselinevars)

# construct dataframe: n, p, q, time, memory, allocations
benchmarkdf = DataFrame(n_obs = cat(n_obs, fill(100, lenvars + lenbvars), dims=1),
        n_vars = cat(fill(10, lenobs), n_vars, fill(10, lenbvars), dims=1),
        n_baselinevars = cat(fill(50, lenobs + lenvars), n_baselinevars, dims=1),
        time = fill(0.0, lenobs+lenvars+lenbvars),
        gctime = fill(0.0, lenobs+lenvars+lenbvars),
        memory = fill(0, lenobs+lenvars+lenbvars),
        allocs = fill(0, lenobs+lenvars+lenbvars)
)

# get it to a shared array for distributed computing
benchmarkarray = SharedArrays.SharedMatrix{Float64}(size(Matrix(benchmarkdf)));
benchmarkarray[:,1:3] = Matrix(benchmarkdf)[:,1:3];

@everywhere eval($benchmarkarray);

### Scenario 1: Fixed number of time-dep and baseline variables, varying number of observations 

In [20]:
@sync @distributed for n_ind in 1:lenobs
    # warmup (first run takes longer because of precompilation times and shouldnt be included)
    n_warmup, p_warmup, q_warmup, q_info_warmup = 100, 10, 10, 10
    xs, x_baseline, tvals, group1, group2 = generate_all(n_warmup, p_warmup, q_warmup, q_info_warmup);
    trainingdata = zip(xs, x_baseline, tvals);
    zdim = nODEparams = 2
    m = init_vae(p_warmup, q_warmup, zdim, nODEparams, prob1)
    L = loss_wrapper(m)
    ps = getparams(m)
    opt = ADAM(0.0005)
    for epoch in 1:35
        Flux.train!(L, ps, trainingdata, opt)
    end
    println("warmup done")
    # now start for real
    n, p, q = n_obs[n_ind], 10, 50 
    println("n=$n, p=$p, q=$q")
    q_info = Int(q/5)
    xs, x_baseline, tvals, group1, group2 = generate_all(n, p, q, q_info);
    trainingdata = zip(xs, x_baseline, tvals);
    zdim = nODEparams = 2
    m = init_vae(p, q, zdim, nODEparams, prob1)
    b = @benchmark run_benchmark($trainingdata, $m) samples=1 evals=1
    println("training done")
    row = n_ind
    benchmarkarray[row,4] = b.times[1] # times
    benchmarkarray[row,5] = b.gctimes[1] # gctimes 
    benchmarkarray[row,6] = b.memory # memory 
    benchmarkarray[row,7] = b.allocs # allocations
end

      From worker 8:	warmup done
      From worker 8:	n=5000, p=10, q=50
      From worker 7:	warmup done
      From worker 7:	n=2000, p=10, q=50


      From worker 6:	warmup done
      From worker 6:	n=1000, p=10, q=50


      From worker 4:	warmup done
      From worker 4:	n=250, p=10, q=50


      From worker 2:	warmup done
      From worker 2:	n=50, p=10, q=50


      From worker 5:	warmup done
      From worker 5:	n=500, p=10, q=50


...

      From worker 5:	training done


      From worker 6:	training done


      From worker 7:	training done


      From worker 8:	training done


Task (done) @0x0000000160b7c010

### Scenario 2: Fixed number of observations and baseline variables, varying number of time-dependent variables

In [21]:
@sync @distributed for p_ind in 1:lenvars
    n, p, q = 100, n_vars[p_ind], 50 
    println("n=$n, p=$p, q=$q")
    q_info = Int(q/5)
    xs, x_baseline, tvals, group1, group2 = generate_all(n, p, q, q_info);
    trainingdata = zip(xs, x_baseline, tvals);
    zdim = nODEparams = 2
    m = init_vae(p, q, zdim, nODEparams, prob1)
    b = @benchmark run_benchmark($trainingdata, $m) samples=1 evals=1
    println("training done")
    row = lenobs + p_ind
    benchmarkarray[row,4] = b.times[1] # times
    benchmarkarray[row,5] = b.gctimes[1] # gctimes 
    benchmarkarray[row,6] = b.memory # memory 
    benchmarkarray[row,7] = b.allocs # allocations
end

      From worker 3:	n=100, p=200, q=50
      From worker 2:	n=100, p=100, q=50


      From worker 10:	n=100, p=20, q=50
      From worker 11:	n=100, p=50, q=50
      From worker 9:	n=100, p=10, q=50


      From worker 2:	training done


      From worker 3:	training done


      From worker 9:	training done


      From worker 10:	training done


      From worker 11:	training done


Task (done) @0x000000015c8a8cd0

### Scenario 3: Fixed number of observations and time-dependent variables, varying number of baseline variables

In [22]:
@sync @distributed for q_ind in 1:lenbvars
    n, p, q = 100, 10, n_baselinevars[q_ind]
    println("n=$n, p=$p, q=$q")
    q_info = Int(q/5)
    xs, x_baseline, tvals, group1, group2 = generate_all(n, p, q, q_info);
    trainingdata = zip(xs, x_baseline, tvals);
    zdim = nODEparams = 2
    m = init_vae(p, q, zdim, nODEparams, prob1)
    b = @benchmark run_benchmark($trainingdata, $m) samples=1 evals=1
    println("training done")
    row = lenobs + lenvars + q_ind
    benchmarkarray[row,4] = b.times[1] # times
    benchmarkarray[row,5] = b.gctimes[1] # gctimes 
    benchmarkarray[row,6] = b.memory # memory 
    benchmarkarray[row,7] = b.allocs # allocations
end

      From worker 8:	n=100, p=10, q=200
      From worker 7:	n=100, p=10, q=100


      From worker 5:	n=100, p=10, q=20
      From worker 6:	n=100, p=10, q=50
      From worker 4:	n=100, p=10, q=10


      From worker 4:	training done


      From worker 5:	training done


      From worker 6:	training done


      From worker 7:	training done


      From worker 8:	training done


Task (done) @0x000000015d685000

## Save results 

First, we can optionally save the Julia object as `JLD2` file:

In [23]:
# if desired: save as JLD2 file 
using JLD2 
JLD2.@save "benchmarkresults.jld2" benchmarkarray
# and re-load from saved
JLD2.@load "benchmarkresults.jld2" 
benchmarkarray = eval(:benchmarkarray)

17×7 SharedMatrix{Float64}:
   50.0   10.0   50.0  1.01433e10  2.93438e8   1.69195e9   1.74255e7
  100.0   10.0   50.0  1.73471e10  5.37488e8   3.38413e9   3.48681e7
  250.0   10.0   50.0  3.82571e10  1.27961e9   8.46083e9   8.72037e7
  500.0   10.0   50.0  6.55462e10  2.26986e9   1.69226e10  1.74469e8
 1000.0   10.0   50.0  1.2202e11   4.36639e9   3.38457e10  3.48972e8
 2000.0   10.0   50.0  1.92253e11  8.15655e9   6.76973e10  6.97948e8
 5000.0   10.0   50.0  4.29818e11  2.05297e10  1.6924e11   1.74553e9
  100.0   10.0   50.0  1.03624e10  4.07309e8   3.38414e9   3.48682e7
  100.0   20.0   50.0  1.14628e10  4.50933e8   3.92182e9   4.46941e7
  100.0   50.0   50.0  1.4565e10   7.2731e8    5.93587e9   7.40914e7
  100.0  100.0   50.0  4.22082e10  3.40051e9   1.06393e10  1.23093e8
  100.0  200.0   50.0  6.55702e10  1.04543e10  2.505e10    2.21107e8
  100.0   10.0   10.0  1.39213e10  4.64147e8   3.30699e9   3.4867e7
  100.0   10.0   20.0  1.42164e10  4.57695e8   3.31784e9   3.48601e7
  100.0

Now, we copy back the information from the `SharedArray` object to the benchmark dataframe, to export that to CSV format.

In [24]:
# copy back to dataframe, to be saved later as CSV
benchmarkdf[:,:time] = benchmarkarray[:,4]
benchmarkdf[:,:gctime] = benchmarkarray[:,5]
benchmarkdf[:,:memory] = benchmarkarray[:,6]
benchmarkdf[:,:allocs] = benchmarkarray[:,7]

17-element Vector{Float64}:
 1.7425528e7
 3.4868112e7
 8.7203655e7
 1.74469471e8
 3.48971551e8
 6.97947943e8
 1.745529024e9
 3.4868182e7
 4.4694106e7
 7.4091394e7
 1.23092985e8
 2.21107247e8
 3.4867032e7
 3.4860097e7
 3.4868112e7
 3.4863294e7
 3.4914004e7

Additionally, we turn the time and memory information into human-readable format and units: 

In [25]:
benchmarkdf[:,:time_in_seconds] = round.(benchmarkarray[:,4] .* 1e-9, digits=3)

# turn memory into human-readable format (taken from BenchmarkTools.jl source code)
benchmarkdf[:,:prettymemory] = prettymemory.(benchmarkarray[:,6])

17-element Vector{String}:
 "1.58 GiB"
 "3.15 GiB"
 "7.88 GiB"
 "15.76 GiB"
 "31.52 GiB"
 "63.05 GiB"
 "157.62 GiB"
 "3.15 GiB"
 "3.65 GiB"
 "5.53 GiB"
 "9.91 GiB"
 "23.33 GiB"
 "3.08 GiB"
 "3.09 GiB"
 "3.15 GiB"
 "3.36 GiB"
 "4.17 GiB"

Finally, we can export to CSV; re-creating Tables 1-3 from the manuscript.

In [26]:
# save entire dataframe as CSV
using CSV 
CSV.write("benchmarkresults.csv", benchmarkdf)

# extract tables as in manuscript appendix and save as CSV files

# different number of observations for fixed p (10) and q (50)
rows_obs = findall(x -> x.n_vars == 10 && x.n_baselinevars == 50, eachrow(benchmarkdf))
times_obs = benchmarkdf[rows_obs, [:n_obs, :time_in_seconds, :prettymemory]]
CSV.write("benchmarkresults_obs.csv", times_obs)

rows_vars = findall(x -> x.n_obs == 100 && x.n_baselinevars == 50, eachrow(benchmarkdf))
times_vars = benchmarkdf[rows_vars, [:n_vars, :time_in_seconds, :prettymemory]]
CSV.write("benchmarkresults_vars.csv", times_vars)

rows_bvars = findall(x -> x.n_obs == 100 && x.n_vars == 10, eachrow(benchmarkdf))
times_bvars = benchmarkdf[rows_bvars, [:n_baselinevars, :time_in_seconds, :prettymemory]]
CSV.write("benchmarkresults_baselinevars.csv", times_bvars)

"benchmarkresults_baselinevars.csv"